In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import unidecode
from pathlib import Path

In [ ]:
# --- Beállítások ---
DATA_PATH = Path("/Users/zelenyianszkimate/Documents/Szakdolgozat/processed_data/processed_data.csv")

# --- Adat betöltése ---
print("Adatok betöltése...")
df = pd.read_csv(DATA_PATH)
print(f"{len(df)} dokumentum betöltve.")

# --- 1. Dokumentumok általános statisztikája ---
print("\nÁltalános statisztikák:")
print(df.describe(include='all'))

# --- 2. Szöveghossz statisztika ---
df['text_length'] = df['text'].fillna('').apply(len)

print("\nSzöveghossz statisztikák:")
print(df['text_length'].describe())

# Szöveghossz eloszlás vizualizáció
plt.figure(figsize=(10,6))
sns.histplot(df['text_length'], bins=50, kde=True)
plt.title('Szöveghossz eloszlása (karakterekben)')
plt.xlabel('Szöveghossz (karakter)')
plt.ylabel('Dokumentumok száma')
plt.grid(True)
plt.show()

# --- 3. Hiányzó metaadatok ---
print("\nHiányzó értékek oszloponkként:")
print(df.isnull().sum())

# --- 4. Dokumentumok megoszlása bíróság szerint ---
if 'birosag' in df.columns:
    plt.figure(figsize=(10,6))
    df['birosag'].value_counts().head(20).plot(kind='bar')
    plt.title('Legnagyobb bírósági dokumentumszámok (Top 20)')
    plt.xlabel('Bíróság')
    plt.ylabel('Dokumentumok száma')
    plt.xticks(rotation=75)
    plt.grid(True)
    plt.show()

# --- 5. Jogszabály hivatkozások elemzése (ha van norm_cite() eredmény) ---
if 'text' in df.columns:
    import re

    LAW_PAT = re.compile(r'([A-ZÁÉÍÓÖŐÚÜŰ]+)_\d+')

    all_laws = []
    for text in df['text']:
        matches = LAW_PAT.findall(str(text))
        all_laws.extend(matches)

    law_counts = pd.Series(all_laws).value_counts()

    print("\nLeggyakoribb jogszabály hivatkozások:")
    print(law_counts.head(10))

    plt.figure(figsize=(10,6))
    law_counts.head(10).plot(kind='bar')
    plt.title('Leggyakoribb jogszabályok (Top 10)')
    plt.xlabel('Jogszabály')
    plt.ylabel('Előfordulás')
    plt.grid(True)
    plt.show()

# --- 6. Rövid szövegek szűrése ---
short_texts = df[df['text_length'] < 50]
print(f"\n{len(short_texts)} db 50 karakternél rövidebb dokumentum található.")

if not short_texts.empty:
    print(short_texts[['doc_id', 'text']].head(10))

# --- 7. Duplikált dokumentumok vizsgálata ---
print("\nDuplikált szövegek keresése:")
duplicates = df[df.duplicated(subset='text')]
print(f"{len(duplicates)} duplikált szöveg található.")

if not duplicates.empty:
    print(duplicates[['doc_id', 'text']].head(5))

print("\nEDA futás befejezve.")
